<a href="https://colab.research.google.com/github/neogeolee/Deeplearning/blob/master/Chatbot_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AEhUxmHkrpho2Ey51bPVilY9NOFDHFWuIOg2DTb75P8J13AKeefBTk
Mounted at /content/drive


In [ ]:
# 필요패키지 임포트
import codecs
import tensorflow as tf
import keras
import numpy as np
import random, sys
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from bs4 import BeautifulSoup

In [ ]:
model = load_model('/content/drive/My Drive/Tensorflow_works/model/chatbot.model')
model

In [ ]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split(',')
      data = data[0] + '' + data[1] + '\n'
      result.append(data)

  result = result[1:] # header정보 제외
  return result


In [ ]:
# 데이터 로드
chat_dataset = load_data('/content/drive/My Drive/Tensorflow_works/data/ChatbotData.csv')
text = ' '.join(chat_dataset)
print(text)

12시 땡!하루가 또 가네요.
 1지망 학교 떨어졌어위로해 드립니다.
 3박4일 놀러가고 싶다여행은 언제나 좋죠.
 3박4일 정도 놀러가고 싶다여행은 언제나 좋죠.
 PPL 심하네눈살이 찌푸려지죠.
 SD카드 망가졌어다시 새로 사는 게 마음 편해요.
 SD카드 안돼다시 새로 사는 게 마음 편해요.
 SNS 맞팔 왜 안하지ㅠㅠ잘 모르고 있을 수도 있어요.
 SNS 시간낭비인 거 아는데 매일 하는 중시간을 정하고 해보세요.
 SNS 시간낭비인데 자꾸 보게됨시간을 정하고 해보세요.
 SNS보면 나만 빼고 다 행복해보여자랑하는 자리니까요.
 가끔 궁금해그 사람도 그럴 거예요.
 가끔 뭐하는지 궁금해그 사람도 그럴 거예요.
 가끔은 혼자인게 좋다혼자를 즐기세요.
 가난한 자의 설움돈은 다시 들어올 거예요.
 가만 있어도 땀난다땀을 식혀주세요.
 가상화폐 쫄딱 망함어서 잊고 새출발 하세요.
 가스불 켜고 나갔어빨리 집에 돌아가서 끄고 나오세요.
 가스불 켜놓고 나온거 같아빨리 집에 돌아가서 끄고 나오세요.
 가스비 너무 많이 나왔다.다음 달에는 더 절약해봐요.
 가스비 비싼데 감기 걸리겠어따뜻하게 사세요!
 가스비 장난 아님다음 달에는 더 절약해봐요.
 가장 확실한 건 뭘까?가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.
 가족 여행 가기로 했어온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 여행 고고온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 여행 어디로 가지?온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족 있어?"저를 만들어 준 사람을 부모님
 가족관계 알려 줘"저를 만들어 준 사람을 부모님
 가족끼리 여행간다.더 가까워질 기회가 되겠네요.
 가족들 보고 싶어저도요.
 가족들이랑 서먹해다들 바빠서 이야기할 시간이 부족했나봐요.
 가족들이랑 서먹해졌어다들 바빠서 이야기할 시간이 부족했나봐요.
 가족들이랑 어디 가지?온 가족이 모두 마음에 드는 곳으로 가보세요.
 가족들이랑 여행 갈거야좋은 생각이에요.
 가족여행

In [ ]:
# 문자 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
voca_size = len(tokenizer.word_index) + 1

print(tokenizer.word_index)

{'거예요': 1, '수': 2, '너무': 3, '좋아하는': 4, '거': 5, '더': 6, '것': 7, '안': 8, '잘': 9, '게': 10, '같아요': 11, '좀': 12, '있어요': 13, '많이': 14, '있을': 15, '사람': 16, '사람이': 17, '마세요': 18, '나': 19, '건': 20, '해보세요': 21, '좋은': 22, '내가': 23, '내': 24, '썸': 25, '어떻게': 26, '왜': 27, '있는': 28, '다': 29, '수도': 30, '이제': 31, '좋을': 32, '다시': 33, '마음이': 34, '것도': 35, '하는': 36, '하고': 37, '또': 38, '오늘': 39, '시간이': 40, '정말': 41, '자꾸': 42, '될': 43, '다른': 44, '없어요': 45, '그': 46, '할': 47, '이별': 48, '바랄게요': 49, '같이': 50, '걸': 51, '돼요': 52, '헤어진지': 53, '좋아요': 54, '한': 55, '보세요': 56, '여자친구가': 57, '남자친구가': 58, '때': 59, '않아요': 60, '드세요': 61, '나를': 62, '진짜': 63, '없는': 64, '마음을': 65, '좋죠': 66, '하세요': 67, '먹고': 68, '못': 69, '먼저': 70, '바랍니다': 71, '계속': 72, '일이': 73, '해요': 74, '말해보세요': 75, '혼자': 76, '생각해요': 77, '뭐': 78, '좋겠어요': 79, '연락': 80, '있을까': 81, '있으면': 82, '몰라요': 83, '이렇게': 84, '이': 85, '사람은': 86, '그냥': 87, '힘든': 88, '사랑이': 89, '연애': 90, '보고': 91, '하면': 92, '마음': 93, '나만': 94, '생각해보세요': 95, '될까': 96, '어떨까요': 97, '시간': 98, '사

In [ ]:
# 문장생성 함수정의
def sentence_generate(model, question):
  init_word = question
  sentence = ''

  for _ in range(3):

    encoded = tokenizer.texts_to_sequences([question])[0]
    encoded = pad_sequences([encoded], maxlen=20, padding='pre')

    result = model.predict_classes(encoded, verbose=0)

    for word, index in tokenizer.word_index.items():
      if index == result:
        break

    question = question + ' ' + word
    sentence = sentence + ' ' + word

  return sentence

In [ ]:
sentence_generate(model, "강아지 키우고 싶어.")

' 남자는 즐거워요 잘'